<a href="https://colab.research.google.com/github/mmehtafenil/Car-Brand-Classification/blob/main/Car_Brand_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from google.colab import drive
drive.mount('/content/drive')
!unrar x '/content/drive/My Drive/Data Science Projects/Car Brand Classification/Datasets.rar' '/content/Dataset/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/My Drive/Data Science Projects/Car Brand Classification/Datasets.rar


Would you like to replace the existing file /content/Dataset/Datasets/Test/audi/21.jpg
 12600 bytes, modified on 2020-07-01 10:35
with a new one
 12600 bytes, modified on 2020-07-01 10:35

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit N


Would you like to replace the existing file /content/Dataset/Datasets/Test/audi/22.jpg
  6483 bytes, modified on 2020-07-01 10:35
with a new one
  6483 bytes, modified on 2020-07-01 10:35

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit A

Extracting  /content/Dataset/Datasets/Test/audi/22.jpg                     1%  OK 
Extracting  /content/Dataset/Datasets/Test/audi/23.jpg                     2%  OK 
Extracting  /content/Dataset/Datasets/T

In [29]:
IMAGE_SIZE = [224, 224]

train_path = '/content/Dataset/Datasets/Train'
valid_path = '/content/Dataset/Datasets/Test'

In [30]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import *
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from glob import glob

In [31]:
resnet = ResNet50(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)


In [32]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [33]:
for layer in resnet.layers:
  layer.trainable = False

In [34]:
folders = glob('/content/Dataset/Datasets/Train/*')

In [35]:
len(folders)

3

In [36]:
x = Flatten()(resnet.output)

In [37]:
prediction = Dense(len(folders), activation = 'softmax')(x)

model = Model(resnet.input, prediction)

In [38]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [39]:
model.compile(
    loss = 'categorial_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [40]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [41]:
training_set = train_datagen.flow_from_directory('/content/Dataset/Datasets/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 64 images belonging to 3 classes.


In [42]:
test_set = test_datagen.flow_from_directory('/content/Dataset/Datasets/Test',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 58 images belonging to 3 classes.


In [46]:
r = model.fit(
    training_set,
    validation_data = test_set,
    epochs = 50,
    steps_per_epoch = len(training_set),
    validation_steps = len(test_set)
)

Epoch 1/50


ValueError: ignored